# Project 1 Design Tradeoffs in Digital Systems

### E1 Chroma up-sampling, YUV pixel manipulation, YUV-RGB CSC

#### 1.a Read the YUV 4:2:0 video sequence(s), and upscale it to 4:4:4

In [1]:
import numpy as np

def read_yuv420_video(yuv_file_path: str, width: int, height: int, num_frames: int) -> (list, list, list):
    """
    Read YUV420 video file, and return pixel data of the three components Y, U, and V.
    Args:
        yuv_file_path (str): Path of the YUV420 video file.
        width (int): Width of the video.
        height (int): Height of the video.
        num_frames (int): Number of frames to read.
    Returns:
        y_data (list): A list of pixel data for the brightness (Y) component
        cb_data (list): The list of pixel data for the chroma (Cb) component
        cr_data (list): The list of pixel data for the chroma (Cr) component
    """
    # initialize the three lists of pixel data
    y_data = []
    cb_data = []
    cr_data = []

    with open(yuv_file_path, 'rb') as file:
        for _ in range(num_frames):
            # read the Y component
            y_frame = np.fromfile(file, dtype=np.uint8, count=width * height).reshape((height, width))
            y_data.append(y_frame)

            # read the U component
            cb_frame = np.fromfile(file, dtype=np.uint8, count=(width // 2) * (height // 2)).reshape((height // 2, width // 2))
            cb_data.append(cb_frame)

            # read the V component
            cr_frame = np.fromfile(file, dtype=np.uint8, count=(width // 2) * (height // 2)).reshape((height // 2, width // 2))
            cr_data.append(cr_frame)

    file.close()

    return y_data, cb_data, cr_data


def upscale_420_to_444(y_data: list, cb_data: list, cr_data: list) -> list:
    """
    Scale 4:2:0 pixel data to 4:4:4 pixel data.
    Args:
        y_data (list): A list of pixel data for the brightness (Y) component
        cb_data (list): The list of pixel data for the chroma (Cb) component
        cr_data (list): The list of pixel data for the chroma (Cr) component
    Returns:
        yuv444_data (list): A list of pixel data for the YUV444 video.
    """
    # initialize the list of pixel data for the YUV444 video
    yuv444_data = []

    for y_frame, cb_frame, cr_frame in zip(y_data, cb_data, cr_data):
        # upscale the U(cb) and V(cr) component, copy to fill
        cb_upsampled = np.repeat(np.repeat(cb_frame, 2, axis=0), 2, axis=1)
        cr_upsampled = np.repeat(np.repeat(cr_frame, 2, axis=0), 2, axis=1)

        # combine the Y, U, and V component
        yuv444_frame = np.dstack((y_frame, cb_upsampled, cr_upsampled))
        yuv444_data.append(yuv444_frame)

    return yuv444_data


def save_yuv444_video(yuv444_data: list, output_file: str) -> None:
    """
    Save the YUV444 video to a file.
    Args:
        yuv444_data (list): A list of pixel data for the YUV444 video.
        output_file (str): Path of the output file.
    """
    with open(output_file, 'wb') as file:
        for frame in yuv444_data:
            frame.tofile(file)

    file.close()

#### Test for 1.a

In [2]:
input_file = "videos/420/foreman_cif.yuv"
output_file = "videos/444/foreman_cif.yuv"
width = 352
height = 288
num_frames = 300

# read the YUV420 video
y_data, u_data, v_data = read_yuv420_video(input_file, width, height, num_frames)

# upscale the YUV420 video to YUV444
yuv444_data = upscale_420_to_444(y_data, u_data, v_data)

# save the YUV444 video
save_yuv444_video(yuv444_data, output_file)

#### 1.b Convert the YUV 4:4:4 video sequence(s) to RGB

In [11]:
from PIL import Image
import numpy as np

def yuv444_to_rgb(yuv444_data: list, csc_matrix: list) -> list:
    """
    Convert YUV 4:4:4 pixel data to RGB pixel data.
    Args:
        yuv444_data (list): 444yuv pixel data list
        csc_matrix (list): convert matrix
    Returns:
        rgb_data (list): RGB pixel data list
    """
    rgb_data = []

    for yuv_frame in yuv444_data:
        # convert 444yuv pixel data to matrix
        yuv_matrix = np.array(yuv_frame).reshape(-1, 3)

        # convert 444yuv pixel data to 444rgb pixel data
        rgb_matrix = np.dot(yuv_matrix - [16, 128, 128], np.array(csc_matrix).reshape(3, 3).T)

        # convert 444rgb pixel data to 444rgb frame data
        rgb_frame = np.clip(rgb_matrix, 0, 255).astype(np.uint8).reshape(yuv_frame.shape)

        # add rgb frame data to rgb data list
        rgb_data.append(rgb_frame)

    return rgb_data

def save_rgb_images(rgb_data: list, output_prefix: str, stop_frame=300) -> None:
    """
    Convert RGB pixel data to RGB images and save them as .png files.
    Args:
        rgb_data (list): rgb pixel data list
        output_prefix (str): png file prefix
    """
    for i, rgb_frame in enumerate(rgb_data):
        # create pillow image object
        img = Image.fromarray(rgb_frame)

        # save image
        img.save(f"{output_prefix}/{i:03d}.png")
        if i == stop_frame:
            break

#### Test for 1.b

In [12]:
csc_matrix = [1.164, 0, 1.596, 1.164, -0.392, -0.813, 1.164, 2.017, 0]

rgb_data = yuv444_to_rgb(yuv444_data, csc_matrix)

save_rgb_images(rgb_data, "images", 4)